## Avalanche DeFi Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.25e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
print(max_slippage)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

-0.09


## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    'avalanche',
    add_category_assets=['yield-yak','gogopool'],
    df_to_remove=[stablecoins,ctokens,atokens]
    )
liquidity

0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
Dropping gogopool because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
pangolin,2.096522e+00,2.022019e+00,-0.035537,avalanche
joe,1.681086e+00,1.614764e+00,-0.039452,avalanche
gmx,1.905603e-02,1.829871e-02,-0.039741,avalanche
benqi,4.473787e+01,4.130219e+01,-0.076796,avalanche
gogopool,7.547334e-02,6.927196e-02,-0.082166,avalanche
stargate-finance,1.450425e+00,1.318411e+00,-0.091017,avalanche
chainlink,4.983535e-02,3.993811e-02,-0.198599,avalanche
ampleforth,7.118583e-01,5.541836e-01,-0.221497,avalanche
aave,8.882365e-03,6.797023e-03,-0.234773,avalanche


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
gmx,GMX,497007415,0.300000,77,0x62edc0692bd897d2295872a9ffcac5425011c661,avalanche
joe,JOE,203237620,0.300000,77,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,avalanche
benqi,QI,98588192,0.201127,51,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,avalanche
pangolin,PNG,97483497,0.198873,50,0x60781c2586d68229fde47564546784ab3faca982,avalanche


In [5]:
from backtester import BackTester,convert_results_table

bt = BackTester(convert_results_table(results),['avalanche-2'])
chart,_,_ = bt.simulate_with_rebalancing(90,0)

/Users/olivermehr/Documents/GitHub/rebalancing-data/PDI/../backtester.py:55: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  self.raw_data = self.raw_data.pct_change()


In [6]:
chart

In [9]:
_,fig=bt.assess_different_rebalancing_periods([30,90],0)
fig

avalanche-2


/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

